In [1]:
%%time
import requests, re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

print("###  이 프로그램은 고려대학교 도서관의 리소스를 사용합니다. 역시 우리 학교가 짱이야!")


def question_maker():
    cal = ''
    n = 0
    while True:
        query = input(str(n+1)+"번째 검색어를 입력해주세요 : ")
        query_type = input("연구자 이름이면 아무 키나 입력하세요")
        query_bool = input("조건을 AND로 설정하려면 아무 키나 입력하세요")
        cal += "&q=" + query + "&si=" + ("AU" if query_type != '' else "TX") + ("&b" + str(n)+ ("=and" if query_bool != '' else "=or"))*2
        n += 1
        killswitch = input("계속하려면 아무 키나 입력하세요.")
        if killswitch == '':
            if n>2:
                cal = cal[:len(cal)-len(("&b" + str(n)+ ("=and" if query_bool != '' else "=or"))*2)]
                cal += "&b" + str(n-1)+ "=AND"
            break
    return cal
    

query_site = r"https://library.korea.ac.kr/eds/brief/discoveryResult?oi=date2&cpp=50&st=KWRD&commandType=advanced"
page_marker = r"&pn="
question = question_maker()



detail_page_starting_name = r"https://library.korea.ac.kr/eds/detail/"

session = requests.Session()
ID = input('아이디를 입력하시오: ')
password = input('패스워드를 입력하시오: ')
session.auth = (ID, password)
auth = session.post(r"https://library.korea.ac.kr/")

article_title = re.compile("<.+searchTitle.+?>(?P<논문제목>.+?)</a>")
article_type = re.compile("<.+pubtype-icon.+>(?P<논문종류>.+)</span>")
article_detail_address = re.compile('<input type="checkbox" name="data" value="(?P<논문주소>.+)" class="checkbox" title="checkbox">')
article_date = re.compile('<th>Publication Year</th>.+?<td>(?P<발행년도>.+?)</td>', re.DOTALL)

# article_source = re.compile('<.+profileTitleSource.+?><a href=".+?&q=(?P<논문출처>.+?)">')
article_source = re.compile('<.+profileTitleSource.+?><a href=".+?>(?P<논문출처>.+?)</a')
article_source_alt = re.compile('<th>TitleSource</.+?<td>(?P<논문출처2>.+?)</td>', re.DOTALL)


article_authors_contents_block = re.compile('<tr><th>저자</th>(?P<저자블록>.+?)</a></td></tr>', re.DOTALL)
article_authors_separated = re.compile('a href.+?>(?P<저자이름>.+?)<')
        
article_abstract = re.compile('<tr><th>초록</th>.+?>(?P<초록>.+?)</td></tr>', re.DOTALL)



page_number = 1
title, classification, source, authors, date, detail_address, abstract = [], [], [], [], [], [], []


###검색결과 불러오기###
###저자, 분류, 세부 페이지 주소###
while True:
    cal = session.get(query_site + page_marker + str(page_number) + question, timeout=None).text
    time.sleep(0.6)
    if r'<div class="resultList">' not in cal:
        break
    title.extend(article_title.findall(cal))
    classification.extend(article_type.findall(cal))
    detail_address.extend(article_detail_address.findall(cal))   
    page_number += 1
    print(page_number-1, "실행 완료")

###세부 페이지 불러오기###
###출처, 날짜, 초록###
while True:
    session = requests.Session()
    session.auth = ("lightsout", password)
    auth = session.post(r"https://library.korea.ac.kr/")
    for i in detail_address:
        cal = session.get(detail_page_starting_name + i, timeout=None).text
        time.sleep(1)
        source_cal = article_source.findall(cal) if article_source.findall(cal) != [] else article_source_alt.findall(cal)
        if source_cal == []:
            source.extend([None])
        else:
            source.extend(source_cal)    
        date_cal = article_date.findall(cal)
        if date_cal == []:
            date.extend([None])
        else:
            date.extend(date_cal)
        authors_cal = article_authors_contents_block.search(cal)
        authors_cal = authors_cal.group() if authors_cal != None else []
        authors.append(article_authors_separated.findall(authors_cal)) if authors_cal != [] else authors.extend([None])
        abstract_cal = article_abstract.findall(cal) if article_abstract.findall(cal) != [] else None
        abstract.extend(abstract_cal) if abstract_cal != None else abstract.extend([None]) 
    else:
        break
        
        
    

###정리해주기###

abstract_original = abstract.copy()

title = [string for string in map(lambda x : re.sub('<br />', ' / ', x), title)]
source = [i for i in map(lambda x: re.sub('<.??i>[.]?', '', x) if x != None else None, source)]

abstract_copy = abstract.copy()
abstract = []
for i in abstract_copy:
    if i==None:
        abstract.append(None)
    else:
        i = re.sub("\s{2,}", " ", i)
        i = re.sub("<.+?>", "", i, re.DOTALL)
        i = i.strip()
        abstract.append(i)


authors_copy = authors.copy()
def list_to_string(iterable):
    import re
    if iterable == None:
        return iterable
    cal = ''
    for i in iterable:
        cal += (i + ', ')
    else:
        cal = cal[:-2]
    return cal

authors = [i for i in map(list_to_string, authors_copy)]
        

###누락 부분 체워넣을 수 있으면 체워넣기###
###년도###
n = 0
date_cal = []
for i in date:
    if i == None:
        cal = re.search('\d{4}', source[n]) if source[n] != None else None
        if cal == None:
            date_cal.append(None)
        else:
            if int(cal.group()) in range(1850, 2500):
                date_cal.append(cal.group())
            else:
                date_cal.append(None)
    else:
        date_cal.append(i)
    n += 1
date = date_cal


result = pd.DataFrame({
    "제목": title,
    "종류": classification,
    "저자": authors,
    "출처": source,
    "년도": date,
    "초록": abstract,
    "주소": detail_address
})


result = result.sort_values("년도")
result.index = range(0, len(result))

###  이 프로그램은 고려대학교 도서관의 리소스를 사용합니다. 역시 우리 학교가 짱이야!
1번째 검색어를 입력해주세요 : 남호성
연구자 이름이면 아무 키나 입력하세요
조건을 AND로 설정하려면 아무 키나 입력하세요
계속하려면 아무 키나 입력하세요.
1 실행 완료
2 실행 완료
3 실행 완료
4 실행 완료
5 실행 완료
6 실행 완료
Wall time: 5min 47s


In [2]:
result.to_excel("남호성.xlsx")

In [3]:
result

,제목,종류,저자,출처,년도,초록,주소
0,학술발표회 초록 : ( 재료 (3) ) : 강유전성 액정의 제조 및 물성에 관한 연...,저널,남호성(Ho Seong Nam); 김성목(Sung Mog Kim); 김종득(Jong...,"한국화학공학회 추계학술발표논문집. Vol. 1992 (1992) pg. 61, 1 p.",1992,None,edskis_kis130755
1,학술발표회 초록 : ( 재료 (3) ) : 강유전성 액정의 제조 및 물성에 관한 연...,저널,남호성(Ho Seong Nam); 김성목(Sung Mog Kim); 김종득(Jong...,"한국화학공학회 추계학술발표논문집. Vol. 1992 (1992) pg. 61, 1 p.",1992,None,edskis_kis130755
2,학술발표회 초록 : ( 재료 (3) ) : 강유전성 액정의 제조 및 물성에 관한 연...,저널,남호성(Ho Seong Nam); 김성목(Sung Mog Kim); 김종득(Jong...,"한국화학공학회 추계학술발표논문집. Vol. 1992 (1992) pg. 61, 1 p.",1992,None,edskis_kis130755
3,학술발표회 초록 : ( 재료 (3) ) : 강유전성 액정의 제조 및 물성에 관한 연...,저널,남호성(Ho Seong Nam); 김성목(Sung Mog Kim); 김종득(Jong...,"한국화학공학회 추계학술발표논문집. Vol. 1992 (1992) pg. 61, 1 p.",1992,None,edskis_kis130755
4,Photon beam과 Electron beam을 병용한 Scalp의 방사선 치료시...,학술지논문,"명노봉, 남호성, 김현, 문봉기",大韓放射線治療技術學會誌 / The journal of the korean radio...,1998,None,edskst_edskst.JAKO199804704297122:JAKO
5,Photon beam과 Electron beam을 병용한 Scalp의 방사선 치료시...,학술지논문,"명노봉, 남호성, 김현, 문봉기",大韓放射線治療技術學會誌 / The journal of the korean radio...,1998,None,edskst_edskst.JAKO199804704297122:JAKO
6,Photon beam과 Electron beam을 병용한 Scalp의 방사선 치료시...,학술지논문,"명노봉, 남호성, 김현, 문봉기",大韓放射線治療技術學會誌 / The journal of the korean radio...,1998,None,edskst_edskst.JAKO199804704297122:JAKO
7,Photon beam과 Electron beam을 병용한 Scalp의 방사선 치료시...,학술지논문,"명노봉, 남호성, 김현, 문봉기",大韓放射線治療技術學會誌 / The journal of the korean radio...,1998,None,edskst_edskst.JAKO199804704297122:JAKO
8,적응 알고리즘을 이용한 DS/CDMA 폐쇄루프 전력제어의 성능분석 / Perform...,Conference,"남호성, 감두열, 박상규","한국통신학회 학술대회 및 강연회 1999-04 :693-696"">한국통신학회 학술대...",1999,None,edspia_edspia.2184460
9,적응 알고리즘을 이용한 DS/CDMA 폐쇄루프 전력제어의 성능분석 / Perform...,Conference,"남호성, 감두열, 박상규","한국통신학회 학술대회 및 강연회 1999-04 :693-696"">한국통신학회 학술대...",1999,None,edspia_edspia.2184460
